<a href="https://colab.research.google.com/github/mohammadRahimi1993/Exercise/blob/main/NodeClassification_DGL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Node Classification with DGL**

1- **Install DGL And Torch**

In [1]:
# https://colab.research.google.com/github/dmlc/dgl/blob/master/notebooks/sparse/quickstart.ipynb#scrollTo=19UZd7wyWzpT
!pip install torch==2.0.0+cpu torchvision==0.15.1+cpu torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cpu > /dev/null
!pip install dgl==1.1.0 -f https://data.dgl.ai/wheels/repo.html > /dev/null

2- **Check dgl and torch**

In [2]:
import os
import torch

os.environ["TORCH"] = torch.__version__
os.environ["DGLBACKEND"] = "pytorch"

try:
  import dgl.sparse as dglsp
  installed = True
except ImportError:
  installed = False

print("installed" if installed else "not installed!")

installed


3- **Import library**

In [3]:
import torch
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

4- **Download CoraGraphDataset**

In [4]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of Categories: {dataset.num_classes}")
print(f"Number of Labels: {dataset.num_labels}")

Extracting file to /root/.dgl/cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of Categories: 7
Number of Labels: 7


/usr/local/lib/python3.10/dist-packages/dgl/data/utils.py:354: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn(


6- **Defining a Graph Convolutional Network (GCN)**

In [6]:
from dgl.nn import GraphConv

class GCN(nn.Module):
  def __init__(self , in_feats, h_feats, num_classes ):
    super(GCN, self).__init__()
    self.conv1 = GraphConv(in_feats, h_feats)
    self.conv2 = GraphConv(h_feats, num_classes)

  def forward(self, g, in_feat):
    h = self.conv1(g, in_feat)
    h = F.relu(h)
    h = self.conv2(g, h)
    return h

model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

In [5]:
g = dataset[0]

print("Edge feature")
print(g.ndata)
print("Node feature")
print(g.edata)

Edge feature
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Node feature
{}


7- **Training the GCN**

In [10]:
def train(g, model):
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  best_val_acc = 0
  best_test_acc = 0

  featurs = g.ndata["feat"]
  labels = g.ndata["label"]
  train_mask = g.ndata["train_mask"]
  val_mask = g.ndata["val_mask"]
  test_mask = g.ndata["test_mask"]

  for e in range(100):
    logits = model(g, featurs)

    # Compute prediction
    pred = logits.argmax(1)

    # Compute loss
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])

    train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
    val_acc =   (pred[val_mask] == labels[val_mask]).float().mean()
    test_acc =  (pred[test_mask] == labels[test_mask]).float().mean()

    if best_val_acc < val_acc:
      best_val_acc = val_acc
      best_val_test = test_acc

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e%5 == 0:
      print(f"In epoch {e}, loss:{loss:.3f}, val_acc:{val_acc:.3f} (best_val_acc {best_val_acc:.3f} , test_acc:{test_acc:.3f}) best_test_acc { best_val_test:.3f}")

model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss:1.946, val_acc:0.208 (best_val_acc 0.208 , test_acc:0.217) best_test_acc 0.217
In epoch 5, loss:1.887, val_acc:0.526 (best_val_acc 0.552 , test_acc:0.536) best_test_acc 0.551
In epoch 10, loss:1.800, val_acc:0.596 (best_val_acc 0.596 , test_acc:0.594) best_test_acc 0.594
In epoch 15, loss:1.688, val_acc:0.652 (best_val_acc 0.652 , test_acc:0.646) best_test_acc 0.646
In epoch 20, loss:1.555, val_acc:0.700 (best_val_acc 0.700 , test_acc:0.685) best_test_acc 0.685
In epoch 25, loss:1.403, val_acc:0.714 (best_val_acc 0.714 , test_acc:0.702) best_test_acc 0.702
In epoch 30, loss:1.237, val_acc:0.718 (best_val_acc 0.720 , test_acc:0.721) best_test_acc 0.717
In epoch 35, loss:1.067, val_acc:0.732 (best_val_acc 0.732 , test_acc:0.733) best_test_acc 0.733
In epoch 40, loss:0.901, val_acc:0.732 (best_val_acc 0.734 , test_acc:0.741) best_test_acc 0.737
In epoch 45, loss:0.747, val_acc:0.744 (best_val_acc 0.744 , test_acc:0.752) best_test_acc 0.749
In epoch 50, loss:0.611, val_acc